# Functions

Topics:
1. How to declare a function
2. Duck-typing in Julia
3. Mutating vs. non-mutating functions
4. Some higher order functions

## How to declare a function
Julia gives us a few different ways to write a function. The first requires the `function` and `end` keywords

In [1]:
function sayhi(name)
    println("Hi $name, it's great to see you!")
end

sayhi (generic function with 1 method)

In [2]:
function f(x)
    x^2
end

f (generic function with 1 method)

We can call either of these functions like this:

In [3]:
sayhi("C-3PO")

Hi C-3PO, it's great to see you!


In [4]:
f(42)

1764

Alternatively, we could have declared either of these functions in a single line

In [5]:
sayhi2(name) = println("Hi $name, it's great to see you!")

sayhi2 (generic function with 1 method)

In [6]:
f2(x) = x^2

f2 (generic function with 1 method)

In [7]:
sayhi2("R2D2")

Hi R2D2, it's great to see you!


In [8]:
f2(42)

1764

Finally, we could have declared these as "anonymous" functions

In [9]:
sayhi3 = name -> println("Hi $name, it's great to see you!")

#9 (generic function with 1 method)

In [10]:
f3 = x -> x^2

#11 (generic function with 1 method)

In [11]:
sayhi3("Chewbacca")

Hi Chewbacca, it's great to see you!


In [12]:
f3(42)

1764

## Duck-typing in Julia
*"If it quacks like a duck, it's a duck."* <br><br>
Julia functions will just work on whatever inputs make sense. <br><br>
For example, `sayhi` works on the name of this minor tv character, written as an integer...

In [13]:
sayhi(55595472)

Hi 55595472, it's great to see you!


And `f` will work on a matrix. 

In [14]:
A = rand(3, 3)
A

3×3 Matrix{Float64}:
 0.850643  0.695355    0.286851
 0.513725  0.00147596  0.490098
 0.839467  0.212896    0.0262699

In [15]:
f(A)

3×3 Matrix{Float64}:
 1.32162   0.653595  0.592336
 0.849176  0.461563  0.160961
 0.845509  0.589635  0.345832

`f` will also work on a string like "hi" because `*` is defined for string inputs as string concatenation.

In [16]:
f("hi")

"hihi"

On the other hand, `f` will not work on a vector. Unlike `A^2`, which is well-defined, the meaning of `v^2` for a vector, `v`, is not a well-defined algebraic operation. 

In [17]:
v = rand(3)

3-element Vector{Float64}:
 0.5872097528033304
 0.46071107674651124
 0.10788351325004975

In [18]:
# This won't work
f(v)

MethodError: MethodError: no method matching ^(::Vector{Float64}, ::Int64)

Closest candidates are:
  ^(!Matched::Float32, ::Integer)
   @ Base math.jl:1277
  ^(!Matched::Regex, ::Integer)
   @ Base regex.jl:863
  ^(!Matched::Missing, ::Integer)
   @ Base missing.jl:165
  ...


## Mutating vs. non-mutating functions

By convention, functions followed by `!` alter their contents and functions lacking `!` do not.

For example, let's look at the difference between `sort` and `sort!`.


In [19]:
v = [3, 5, 2]

3-element Vector{Int64}:
 3
 5
 2

In [20]:
sort(v)

3-element Vector{Int64}:
 2
 3
 5

In [21]:
v

3-element Vector{Int64}:
 3
 5
 2

`sort(v)` returns a sorted array that contains the same elements as `v`, but `v` is left unchanged. <br><br>

On the other hand, when we run `sort!(v)`, the contents of v are sorted within the array `v`.

In [22]:
sort!(v)

3-element Vector{Int64}:
 2
 3
 5

In [23]:
v

3-element Vector{Int64}:
 2
 3
 5

## Some higher order functions

### map

`map` is a "higher-order" function in Julia that *takes a function* as one of its input arguments. 
`map` then applies that function to every element of the data structure you pass it. For example, executing

```julia
map(f, [1, 2, 3])
```
will give you an output array where the function `f` has been applied to all elements of `[1, 2, 3]`
```julia
[f(1), f(2), f(3)]
```

In [24]:
map(f, [1, 2, 3])

3-element Vector{Int64}:
 1
 4
 9

Here we've squared all the elements of the vector `[1, 2, 3]`, rather than squaring the vector `[1, 2, 3]`.

To do this, we could have passed to `map` an anonymous function rather than a named function, such as

In [25]:
x -> x^3

#13 (generic function with 1 method)

via

In [26]:
map(x -> x^3, [1, 2, 3])

3-element Vector{Int64}:
  1
  8
 27

and now we've cubed all the elements of `[1, 2, 3]`!

### broadcast

`broadcast` is another higher-order function like `map`. `broadcast` is a generalization of `map`, so it can do every thing `map` can do and more. The syntax for calling `broadcast` is the same as for calling `map`

In [27]:
broadcast(f, [1, 2, 3])

3-element Vector{Int64}:
 1
 4
 9

and again, we've applied `f` (squared) to all the elements of `[1, 2, 3]` - this time by "broadcasting" `f`!

Some syntactic sugar for calling `broadcast` is to place a `.` between the name of the function you want to `broadcast` and its input arguments. For example,

```julia
broadcast(f, [1, 2, 3])
```
is the same as
```julia
f.([1, 2, 3])
```

In [28]:
f.([1, 2, 3])

3-element Vector{Int64}:
 1
 4
 9

Notice again how different this is from calling 
```julia
f([1, 2, 3])
```
We can square every element of a vector, but we can't square a vector!

To drive home the point, let's look at the difference between

```julia
f(A)
```
and
```julia
f.(A)
```
for a matrix `A`:

In [29]:
A = [i + 3*j for j in 0:2, i in 1:3]

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

In [30]:
f(A)

3×3 Matrix{Int64}:
  30   36   42
  66   81   96
 102  126  150

As before we see that for a matrix, `A`,
```
f(A) = A^2 = A * A
``` 

On the other hand,

In [31]:
B = f.(A)

3×3 Matrix{Int64}:
  1   4   9
 16  25  36
 49  64  81

contains the squares of all the entries of `A`.

This dot syntax for broadcasting allows us to write relatively complex compound elementwise expressions in a way that looks natural/closer to mathematical notation. For example, we can write

In [32]:
A .+ 2 .* f.(A) ./ A

3×3 Matrix{Float64}:
  3.0   6.0   9.0
 12.0  15.0  18.0
 21.0  24.0  27.0

instead of

In [33]:
broadcast(x -> x + 2 * f(x) / x, A)

3×3 Matrix{Float64}:
  3.0   6.0   9.0
 12.0  15.0  18.0
 21.0  24.0  27.0

and the two will perform exactly the same.

### Exercises

#### 6.1 
Write a function `add_one` that adds 1 to its input.

In [41]:
function add_one(x)
    x = x + 1
end


add_one (generic function with 1 method)

In [42]:
@assert add_one(1) == 2

In [43]:
@assert add_one(11) == 12

#### 6.2 
Use `map` or `broadcast` to increment every element of matrix `A` by `1` and assign it to a variable `A1`.

In [46]:
A1 = add_one.(A)

3×3 Matrix{Int64}:
 2  3   4
 5  6   7
 8  9  10

In [47]:
@assert A1 == [2 3 4; 5 6 7; 8 9 10]

#### 6.3 
Use the broadcast dot syntax to increment every element of matrix `A1` by `1` and store it in variable `A2`

In [48]:
A2 = add_one.(A1)

3×3 Matrix{Int64}:
 3   4   5
 6   7   8
 9  10  11

In [49]:
@assert A2 == [3 4 5; 6 7 8; 9 10 11]